# Replicating Results


### What is this?
This notebook provides step-by-step instructions for replicating the results for multi-adjustment

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

import os
import yaml
import sqlalchemy

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = None

from IPython.display import display 
from itertools import permutations, combinations
from jinja2 import Template
import dateparser


from ohio.ext.numpy import pg_copy_to_table

Specify The Database, demo_col of interest, demo_values of interest and the procedure you wish to use for adjustment

In [2]:
database_directory = "housing_safety"

In [ ]:
notebook_config = None
with open(database_directory+"/config.yaml") as f:
    notebook_config = yaml.full_load(f)

In [ ]:
database = notebook_config["database"]

In [3]:
working_schema = notebook_config['working_schema']

In [4]:
results_schema = notebook_config['results_schema']

In [5]:
demo_col = notebook_config['demo_col']

In [ ]:
demo_values = notebook_config['demo_values']

## Getting Set Up

You will need a database with the following schemas:
- **public**: The raw data from donors choose as well as some tables with calculated features and intermediate modeling tables
- **model_metadata**: Information about the models we ran, such as model types and hyperparameters (models were run with `triage`, which generates this schema. In `triage` a "model group" specifies a type of model and associated hyperparameter values, while a "model" is an instantiation of a given model group on a specific temporal validation split). Note that this schema contains information on other model runs with this dataset, in addition to the run used for the current study of fairness-accuracy trade-offs.
- **test_results**: Validation set statistics and predictions for the models. Here, `test_results.predictions` contains project-level predicted scores from each model in the grid, while `test_results.evaluations` contains aggregated summary statistics for each model.
- **train_results**: Training set statistics for the models, including feature importances.
- **features**: Intermediate tables containing calculated features from the `triage` run.
- **bias_working**: Intermediate tables from the bias analysis, as well as the mapping table between projects and school poverty levels, `bias_working.entity_demos`.
- **bias_results_submitted**: Results of the fairness-accuracy trade-offs from the study as submitted (see below to use these to replicate the figures from the study).
- **bias_results**: Empty bias analysis results tables that will be populated by re-running the fairness adjustments (see below for instructions).


Finally, to connect to the database, you'll need a yaml file named `db_profile.yaml` in the same directory as this notebook with your connection info:
```yaml
host: {POSTGRES_HOST}
user: {POSTGRES_USER}
db: education_crowdfunding
pass: {POSTGRES_PASSWORD}
port: {POSTGRES_PORT}
```


In [7]:
with open('../../config/db_default_profile.yaml') as fd:
    config = yaml.full_load(fd)
    dburl = sqlalchemy.engine.url.URL(
        "postgresql",
        host=config["host"],
        username=config["user"],
        database=database,
        password=config["pass"],
        port=config["port"],
    )
    engine = sqlalchemy.create_engine(dburl, poolclass=sqlalchemy.pool.QueuePool)

/tmp/ipykernel_16651/127057796.py:3: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  dburl = sqlalchemy.engine.url.URL(


In [8]:
def get_frac_demos(model_group_id=None, train_end_time=None):
    if model_group_id is None and train_end_time is None:
        sel_string = ""
    elif model_group_id is None:
        sel_string = f"and train_end_time = '{train_end_time}'"
    elif train_end_time is None:
        sel_string = f"and model_group_id = {model_group_id}"
    else:
        sel_string = f"and train_end_time = '{train_end_time}' and model_group_id = {model_group_id}"
    sql = f"""
    select model_group_id, train_end_time, weight, count(distinct entity_id) 
    from {results_schema}.selected_entities se natural join {working_schema}.entity_demos ed
    where ed.{plevel} = '{demo_values[0]}' {sel_string}
    group by 1, 2, 3 
    """
    ts_df = pd.read_sql(sql, engine)
    return ts_df

In [12]:
def get_difference_df(weight1, weight2, model_group_id=None, train_end_time=None):
    if model_group_id is None and train_end_time is None:
        sel_string = f""
    elif model_group_id is None:
        sel_string = f"WHERE train_end_time = '{train_end_time}'"
    elif train_end_time is None:
        sel_string = f"WHERE model_group_id = {model_group_id}"
    else:
        sel_string = f"WHERE model_group_id = {model_group_id} and train_end_time = '{train_end_time}'"
    
    sql = f"""
        with all_entities as (select model_group_id, train_end_time, weight, entity_id, {demo_col} 
        from {results_schema}.selected_entities se natural join {working_schema}.entity_demos ed)
        , e1 as (select * from all_entities where weight = {weight1})
        , e2 as (select * from all_entities where weight = {weight2})
        , e3 as (select * from e1 where (model_group_id , train_end_time , entity_id) not in (select model_group_id , train_end_time , entity_id  from e2))
        , e4 as (select * from e2 where (model_group_id , train_end_time , entity_id) not in (select model_group_id , train_end_time , entity_id  from e1))
        SELECT distinct * from e3 UNION ALL SELECT distinct * FROM e4 {sel_string}
        """    
        
    ts_df = pd.read_sql(sql, engine)
    return ts_df    

In [13]:
diff_df = get_difference_df(weight1=0.9, weight2=0.8)

In [14]:
print(f"Model Group IDs Used")
print(diff_df['model_group_id'].unique())
print(f"Train End Times Used")
print(diff_df['train_end_time'].unique())

Model Group IDs Used
[942]
Train End Times Used
['2012-09-01T00:00:00.000000000' '2011-05-01T00:00:00.000000000'
 '2012-01-01T00:00:00.000000000' '2011-09-01T00:00:00.000000000'
 '2011-11-01T00:00:00.000000000' '2012-05-01T00:00:00.000000000'
 '2013-01-01T00:00:00.000000000' '2012-11-01T00:00:00.000000000'
 '2012-07-01T00:00:00.000000000']


In [20]:
diff_df

,model_group_id,train_end_time,weight,entity_id,plevel
0,942,2012-09-01,0.9,81947,not_highest
1,942,2011-05-01,0.9,540775,highest
2,942,2012-01-01,0.9,348768,not_highest
3,942,2011-09-01,0.9,538589,not_highest
4,942,2012-01-01,0.9,324618,not_highest
5,942,2011-11-01,0.9,49931,not_highest
6,942,2012-01-01,0.9,468993,not_highest
7,942,2012-05-01,0.9,321809,highest
8,942,2013-01-01,0.9,217413,not_highest
9,942,2012-11-01,0.9,137422,highest


# Performing Entity Analysis

Assumes that the model_adjustment_results_{demo_col} table is populated correctly for model selection

In [ ]:
from DJRecallAdjuster import ra_procedure

In [1]:
ra_procedure(directory=database_directory, weights=None, pause_phases=False, entity_selection=True):

NameError: name 'ra_procedure' is not defined